# Trabalho 1
Matéria: Processamento de Linguagem Natural <br>
Dupla/Matrícula: Bianca Nunes Coelho - 15102880, Raíssa Nunes Coelho - 15102887 <br> 

## Introdução

## Bibliotecas
Bibliotecas do ptyhon utilizadas neste trabalho estão a seguir.

In [55]:
import pandas as pd
import re
import nltk
import spacy
import gensim

## Datasets
Datasets utilizados estão definidos a seguir.

In [65]:
file = "train-00000-of-00001-ecf072e7202447d8.parquet"
df = pd.read_parquet(file, engine= 'pyarrow')

In [66]:
len(df)

3877

In [67]:
df.head(4)

,titulo,texto,link,resumo,categoria,data_hora
0,"Veja a trajetória de Fernando Diniz, em busca ...",Se tem um treinador que sempre foi badalado pe...,https://www.cnnbrasil.com.br/esportes/veja-a-t...,Técnico tricolor e da Seleção Brasileira tem a...,esportes,2023-11-04 12:26:18
1,"Após tempestade com ventos, São Paulo terá sáb...",Após uma tempestade com ventos de mais de 100 ...,https://www.cnnbrasil.com.br/nacional/apos-tem...,"Termômetros devem chegar a 26ºC, esquentando a...",nacional,2023-11-04 12:15:44
2,Como os aplicativos de relacionamento “arruina...,Jenny Nguyen-Don estava com vontade de comer f...,https://www.cnnbrasil.com.br/tecnologia/como-o...,Pessoas em busca de relacionamentos afirmam qu...,tecnologia,2023-11-04 12:15:07
3,Fluminense pode ser o 26º clube a levantar a t...,O Fluminense vai em busca do primeiro título d...,https://www.cnnbrasil.com.br/esportes/fluminen...,"Em busca do título inédito, o Tricolor carioca...",esportes,2023-11-04 12:05:24


In [68]:
# pega coluna
text = df['texto']

In [69]:
date = df['data_hora']

## Pré-processamento

In [70]:
for i in range(len(text)):
    text[i] = text[i].lower()

In [71]:
from unidecode import unidecode

for i in range(len(text)):
    text[i] = unidecode(text[i])

In [75]:
for i in range(len(text)):
    text[i] = re.sub(r'[^a-zA-Z\s]', "" ,text[i])

In [76]:
print(text[0])

se tem um treinador que sempre foi badalado pela forma como faz seu time jogar  este cara e fernando diniz  e o comandante do fluminense tem a chance de chegar ao apice da carreira neste sabado      a partir das   h  de brasilia   quando decide a copa libertadores contra o boca juniors arg  no maracana  rumo a gloria eterna  e sua gloria pessoal  considerado ha alguns anos um dos melhores   senao o melhor   tecnico do futebol brasileiro  fernando diniz sempre esbarra na desconfianca que o persegue  uma desconfianca  resultadista   afinal  como pode um treinador tao elogiado ter apenas um campeonato carioca        na bagagem  titulos de expressao e diniz  ate entao  pareciam nao combinar   leia mais  certo e que a final da copa libertadores ja vai alcar o tecnico a outro patamar  mesmo sendo interino na selecao brasileira  conquistar o maior torneio de clubes da america do sul tera um valor inestimavel  alem de entrar na galeria da fama do fluminense  o treinador tricolor dara uma respo

In [77]:
token_text = []

for j in text:
    token_text.append(nltk.word_tokenize(j))

In [78]:
print(token_text[0])

['se', 'tem', 'um', 'treinador', 'que', 'sempre', 'foi', 'badalado', 'pela', 'forma', 'como', 'faz', 'seu', 'time', 'jogar', 'este', 'cara', 'e', 'fernando', 'diniz', 'e', 'o', 'comandante', 'do', 'fluminense', 'tem', 'a', 'chance', 'de', 'chegar', 'ao', 'apice', 'da', 'carreira', 'neste', 'sabado', 'a', 'partir', 'das', 'h', 'de', 'brasilia', 'quando', 'decide', 'a', 'copa', 'libertadores', 'contra', 'o', 'boca', 'juniors', 'arg', 'no', 'maracana', 'rumo', 'a', 'gloria', 'eterna', 'e', 'sua', 'gloria', 'pessoal', 'considerado', 'ha', 'alguns', 'anos', 'um', 'dos', 'melhores', 'senao', 'o', 'melhor', 'tecnico', 'do', 'futebol', 'brasileiro', 'fernando', 'diniz', 'sempre', 'esbarra', 'na', 'desconfianca', 'que', 'o', 'persegue', 'uma', 'desconfianca', 'resultadista', 'afinal', 'como', 'pode', 'um', 'treinador', 'tao', 'elogiado', 'ter', 'apenas', 'um', 'campeonato', 'carioca', 'na', 'bagagem', 'titulos', 'de', 'expressao', 'e', 'diniz', 'ate', 'entao', 'pareciam', 'nao', 'combinar', 'le

In [79]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/raissa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
from nltk.corpus import stopwords
sw = stopwords.words('portuguese')

In [81]:
for stop in sw:
    for text in token_text:
        for t in text:
            if(t == stop):
                text.remove(t)

In [82]:
print(token_text[0])

['treinador', 'sempre', 'badalado', 'forma', 'faz', 'time', 'jogar', 'cara', 'fernando', 'diniz', 'comandante', 'fluminense', 'chance', 'chegar', 'apice', 'carreira', 'neste', 'sabado', 'partir', 'h', 'brasilia', 'decide', 'copa', 'libertadores', 'contra', 'boca', 'juniors', 'arg', 'maracana', 'rumo', 'gloria', 'eterna', 'gloria', 'pessoal', 'considerado', 'ha', 'alguns', 'anos', 'melhores', 'senao', 'melhor', 'tecnico', 'futebol', 'brasileiro', 'fernando', 'diniz', 'sempre', 'esbarra', 'desconfianca', 'persegue', 'desconfianca', 'resultadista', 'afinal', 'pode', 'treinador', 'tao', 'elogiado', 'ter', 'apenas', 'campeonato', 'carioca', 'bagagem', 'titulos', 'expressao', 'diniz', 'ate', 'entao', 'pareciam', 'nao', 'combinar', 'leia', 'certo', 'final', 'copa', 'libertadores', 'ja', 'vai', 'alcar', 'tecnico', 'outro', 'patamar', 'sendo', 'interino', 'selecao', 'brasileira', 'conquistar', 'maior', 'torneio', 'clubes', 'america', 'sul', 'tera', 'valor', 'inestimavel', 'alem', 'entrar', 'gal